# Chapter 2. 파이썬스러운(pythonic) 코드
+ 이 장의 목표
    + 인덱스와 슬라이스를 이해하고 인덱싱 가능한 객체를 올바른 방식으로 구현
    + 시퀀스와 이터러블 구현
    + 매직 메서드를 사요해 보다 관용적인 코드 구현
    + 부작용을 유발하는 흔한 실수 피하기

## 인덱스와 슬라이스

In [1]:
my_numbers = (4, 5, 3, 9)

print("음수 인덱스:", my_numbers[-1], my_numbers[-3])
print("slice 1:", my_numbers[2:5])
print("slice 2:", my_numbers[:3])
print("slice 3:", my_numbers[3:])
print("slice 4:", my_numbers[::-1])

print("\n")
print("slice index:", my_numbers[1:7:2])
print("slice object:", my_numbers[slice(1, 7, 2)])

print("\n")
interval = slice(None, 3)
print("slice object vs index:", my_numbers[interval] == my_numbers[:3])

음수 인덱스: 9 5
slice 1: (3, 9)
slice 2: (4, 5, 3)
slice 3: (9,)
slice 4: (9, 3, 5, 4)


slice index: (5, 9)
slice object: (5, 9)


slice object vs index: True


## 자체 시퀀스 생성
+ 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여아 한다.
+ slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외한다.

In [2]:
""" list wrapping 사례 """

class Item:
    def __init__(self, *values):
        self._value = list(values)
    
    def __len__(self):
        return len(self._value)

    def __getitem__(self, index):
        return self._value.__getitem__(index)

In [3]:
item = Item(3, 4, 5)
print(len(item))
print(item[1:])
print(type(item[1:]))

3
[4, 5]
<class 'list'>


## 컨텍스트 관리자
+ __enter__와 __exit__ 두 개의 매직 메서드로 구성됨
+ with 문은 __enter__ 메서드를 호추랗고 이 메서드가 무엇을 반환하든 as 이후에 지정된 변수에 할당됨
+ 해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 __exit__ 메서드를 호출함
+ 블록 내에 예외 또는 오류가 있는 경우에도 __exit__ 메서드가 여전히 호출되며, 예외는 파라미터로 확인 가능
    + True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파하지 않고 정지함을 의미하나 좋지 않은 습관임

In [4]:
""" 컨텍스트 관리자를 활용한 DB Dump 사례 """

def stop_database():
    print("systemctl stop postgresql.service")

def start_database():
    print("systemctl start postgresql.service")


class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()


def db_backup():
    print("pg_dump database")

def main():
    with DBHandler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


### 컨텍스트 관리자 구현
+ contextlib 모듈을 활용한 컨텍스트 관리자 구현 가능
    + contextlib.contextmanager 데코레이터를 적용하면 해당 함수의 코드를 컨텍스트 관리자로 변환함
    + contextlib.ContextDecorator 를 사용하면 with문 없이 완전히 독립적인 실행가능
        + 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 얻을 수 없는 단점
        + e.g.) with offline_back() as bp: 처럼 사용할 수 없음
    + with contextlib.suppress(DataConversionException) 은 로직 자체적으로 처리하고 있음을 예외임을 명시하고 실패하지 않도록 함

In [5]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

def main():
    with db_handler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [6]:
import contextlib

class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    print("pg_dump database")

## 프로퍼티, 속성과 객체 메서드의 다른 타입들

+ 파이썬 객체의 모든 프로퍼티와 함수는 public
+ 따라서 엄격한 강제사항은 없지만 밑줄로 시작하는 속성은 해당 객체에 대해 private을 의미하며 외부에서 호출하지 않기를 기대함

### 파이썬에서의 밑줄
+ 이중 밑줄은 여러 번 확장되는 클래스의 메서드를 이름 충돌 없이 오버라이드하기 위해 만들어짐
+ 이중 밑줄이 아닌 하나의 밑줄을 사용하는 파이썬의 관습을 지킬 것


In [7]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60 # 하나의 밑줄

conn = Connector("postgresql://localhost")

conn._timeout = 70
print(conn.__dict__)

{'source': 'postgresql://localhost', '_timeout': 70}


In [8]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60 # 두 개의 밑줄

conn = Connector("postgresql://localhost")

conn.__timeout # "접근할 수 없다"가 아닌 "존재하지 않는다"는 오류 발생

AttributeError: 'Connector' object has no attribute '__timeout'

In [9]:
conn._Connector__timeout = 80 # 이름이 바뀌어 있음
print(conn.__dict__)

{'source': 'postgresql://localhost', '_Connector__timeout': 80}


### 프로퍼티

+ 객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용함 (잘못된 정보의 입력 제한 등)
+ 모든 속성에


In [10]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email

user = User("shyeon.kang")
user.email = "shyeon.kang@"

ValueError: 유효한 이메일이 아니므로 shyeon.kang@ 값을 사용할 수 없음

In [11]:
user = User("Kang Seonghyeon")
user.email = "Seonghyeon@gmail.com"

## 이터러블 객체
+ 반복 가능한지 확인하기 위해 파이썬은 고수준에서 다음 두 가지를 차례로 검사
    + 객체가 __next__나 __iter__ 이터레이터 메스드 중 하나를 포함하는지 여부
    + 객체가 시퀀스이고 __len__과 __getitem__을 모두 가졌는지 여부

### 이터러블 객체 만들기
+ 객체를 반복하려고 하면 파이썬은 해당 객체의 iter() 함수를 호출
+ for 문은 StopIteration 예외가 발생할 때까지 next()를 호출하는 것과 같음

In [12]:
from datetime import timedelta
from datetime import date

class DateRangeIterable:
    """ 자체 이터레이터 메서드를 가지고 있음 """

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        return self # 자신이 이터러블임을 나타냄

    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

for day in DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5)):
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


+ for 문은 StopIteration 예외가 발생할 때까지 next()를 호출하는 것과 같음

In [13]:
r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))

In [14]:
next(r), next(r), next(r), next(r)

(datetime.date(2019, 1, 1),
 datetime.date(2019, 1, 2),
 datetime.date(2019, 1, 3),
 datetime.date(2019, 1, 4))

In [15]:
next(r)

StopIteration: 

+ 한 번 실행하면 끝의 날짜에 도달한상태이므로 이후에 호출하면 계속 StopIteration 예외가 발행하는 문제
+ 제너레이터를 사용하면 이러한 문제를 해결할 수 있음(<b>컨테이너 이터러블</b>이라고 함)

In [16]:
r1 = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str, r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [17]:
max(r1)

ValueError: max() arg is an empty sequence

In [18]:
""" 컨테이너 이터러블 적용 """

class DateRangeIterable:
    """ 자체 이터레이터 메서드를 가지고 있음 """

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

In [19]:
r1 = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str, r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [20]:
max(r1)

datetime.date(2019, 1, 4)

### 시퀀스 만들기
+ 이터러블은 메모리를 적게 사용하지만 n번째 요소를 얻기 위한 시각복잡도는 O(n)이다.
+ 시퀀스로 구현하면 더 많은 메모리가 사용되지만(모든 것을 보관해야 하므로) 특정 요소를 가져오기 위한 인덱싱 시간복잡도는 O(1)로 상수에 가능하다.

In [21]:
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()

    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days

    def __getitem__(self, day_no):
        return self._range[day_no]

    def __len__(self):
        return len(self._range)

In [22]:
s1 = DateRangeSequence(date(2019, 1, 1), date(2019, 1, 5))
for day in s1:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [23]:
s1[0], s1[3], s1[-1]

(datetime.date(2019, 1, 1),
 datetime.date(2019, 1, 4),
 datetime.date(2019, 1, 4))

## 컨테이너 객체

+ 컨테이너는 __contain__메서드를 구현한 객체임
+ 일반적으로 Boolean을 반환하며 in 키워드가 발견될 때 호출됨
    + element in container ==> container.__contains__(element)

In [24]:
class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __contains__(self, coord):
        x, y = coord
        return 0 <=x < self.width and 0 <= y < self.height
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)

    def __contains__(self, coord):
        return coord in self.limits

def mark_coordiante(grid, coord):
    if coord in grid:
        print(f"{coord} is in limit")
    else:
        print(f"{coord} is out of the limit")

In [25]:
grid = Grid(4, 5)

coord = (3, 4)
mark_coordiante(grid, coord)

(3, 4) is in limit


In [26]:
grid = Grid(4, 5)

coord = (5, 7)
mark_coordiante(grid, coord)

(5, 7) is out of the limit


## 객체의 동적인 속성

+ 파이썬은 객체를 호출하면 객체 사전에서 <myattribute>를 찾아서 __getattribute__를 호출
+ 객체에 찾고 있는 속성이 없는 경우 객체 이름을 파라미터로 __getattr__ 이라는 추가 메서드가 호출됨

In [27]:
class DynamicAttribute:
    def __init__(self, attribute):
        self.attribute = attribute

    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음")

In [28]:
dyn = DynamicAttribute("value")
dyn.attribute

'value'

In [29]:
dyn.fallback_test

'[fallback resolved] test'

In [30]:
dyn.__dict__["fallback_new"] = "new value"
dyn.fallback_new

'new value'

In [31]:
getattr(dyn, "something", "default")

'default'

In [32]:
getattr(dyn, "fallback_new", "default")

'new value'

In [33]:
dyn.something

AttributeError: DynamicAttribute에는 something 속성이 없음

## 호출형(callable) 객체
+ __call__을 사용하면 객체를 일반 함수처럼 호출할 수 있음
+ 여기에 전달된 모든 파라미터는 __call__ 메서드에 그대로 전달됨

In [34]:
from collections import defaultdict

class CallCount:
    def __init__(self):
        self._counts = defaultdict(int)

    def __call__(self, argument):
        self._counts[argument] += 1
        return self._counts[argument]

In [35]:
cc = CallCount()
print(cc(1))
print(cc(2))
print(cc(2))
print(cc(1))

1
1
2
2


## 파이썬에서 유의할 점

### 변경 가능한 파라미터의 기본 값
+ 변경 가능한 객체를 함수의 기본 인자로 사용하면 안됨

In [36]:
def wrong_user_display(user_metadata: dict = {"name":"John", "age":30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    return f"{name} ({age})"

In [37]:
wrong_user_display()

'John (30)'

In [38]:
wrong_user_display({"name":"kang", "age":20})

'kang (20)'

In [39]:
wrong_user_display()

KeyError: 'name'

In [61]:
""" 함수 내로 기본값 설정 """
def good_user_display(user = None):
    user_metadata = {"name":"John", "age":30} # 기본값?

    def display(user_metadata = user_metadata):
        name = user_metadata.pop("name")
        age = user_metadata.pop("age")
        return f"{name} ({age})"

    return display(user)

In [62]:
good_user_display()

AttributeError: 'NoneType' object has no attribute 'pop'

In [57]:
good_user_display({"name":"kang", "age":20})

'kang (20)'

In [58]:
good_user_display()

AttributeError: 'NoneType' object has no attribute 'pop'

### 내장(built-in) 타입 확장

+ 리스트, 문자열, 사전과 같은 내장 타입을 확장하는 올바른 방법은 collections 모듈을 사용하는 것
+ 내장 함수에서 override가 동작하지 않음

In [44]:
class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}  "

In [45]:
bl = BadList((0, 1, 2, 3, 4, 5))

In [46]:
bl[0]

'[even] 0  '

In [47]:
bl[1]

'[odd] 1  '

In [48]:
"".join(bl) # join() 내부의 __getitem__ 함수가 호출되지 않음

TypeError: sequence item 0: expected str instance, int found

In [49]:
""" collections 모듈을 사용하여 문제 해결 """

from collections import UserList

class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}  "

In [50]:
bl = GoodList((0, 1, 2, 3, 4, 5))
"".join(bl)

'[even] 0  [odd] 1  [even] 2  [odd] 3  [even] 4  [odd] 5  '